In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from pathlib import Path

# optioneel: voor GIF
import imageio.v2 as imageio  

In [2]:
# Moon
X, y = make_moons(n_samples=800, noise=0.2, random_state=0)

In [3]:
# model 
clf = Pipeline([
    ("scaler", StandardScaler()),
    ("mlp", MLPClassifier(
        hidden_layer_sizes=(16,16),
        activation="tanh",
        solver="adam",
        max_iter=1,
        warm_start=True,
        verbose=True, 
        random_state=0
    )),
])
clf

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=(16, 16),
                               max_iter=1, random_state=0, verbose=True,
                               warm_start=True))])

In [4]:
# plot
def plot_decision_boundary(clf, X, y, title, path=None):
    x_min, x_max = X[:, 0].min() - 1.0, X[:, 0].max() + 1.0
    y_min, y_max = X[:, 1].min() - 1.0, X[:, 1].max() + 1.0
    xx, yy = np.meshgrid(
        np.linspace(x_min, x_max, 400),
        np.linspace(y_min, y_max, 400)
    )
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1].reshape(xx.shape)

    plt.figure(figsize=(6,5))
    plt.contourf(xx, yy, Z, alpha=0.6, levels=30)
    plt.scatter(X[y==0,0], X[y==0,1], s=15, label="Class 0")
    plt.scatter(X[y==1,0], X[y==1,1], s=15, label="Class 1")
    plt.title(title)
    plt.xlabel("x1"); plt.ylabel("x2"); plt.legend(loc="best")
    plt.tight_layout()
    if path:
        plt.savefig(path, dpi=120, bbox_inches="tight")
        plt.close()
    else:
        plt.show()

In [5]:
# train
outdir = Path("frames")
outdir.mkdir(exist_ok=True)
n_epochs = 200  #itterations
saved = []

for epoch in range(1, n_epochs+1):
    clf.fit(X, y)  # traint 1 extra epoch
    frame_path = outdir / f"frame_{epoch:04d}.png"
    plot_decision_boundary(clf, X, y, f"Epoch {epoch}", frame_path)
    saved.append(frame_path)

print("Aantal frames:", len(saved))

Iteration 1, loss = 0.68571218


/Users/amusuokamoto/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.64785147


/Users/amusuokamoto/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.61204559


...

Iteration 1, loss = 0.09611249


/Users/amusuokamoto/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Iteration 1, loss = 0.09552106


/Users/amusuokamoto/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(


Aantal frames: 200


In [6]:
# GIF
images = [imageio.imread(p) for p in saved]
gif_path = "training_progress.gif"
imageio.mimsave(gif_path, images, duration=0.2)  # 0.2s per frame
print("GIF opgeslagen als:", gif_path)


GIF opgeslagen als: training_progress.gif
